In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [3]:
mnist = tf.keras.datasets.mnist # load datasets

(x_train, y_train), (x_test, y_test) = mnist.load_data() # load data
x_train, x_test =  x_train / 255.0, x_test / 255.0
print(x_train.shape) # we now have 60000 imgs


(60000, 28, 28)


In [4]:
x_train = x_train[ ... , tf.newaxis].astype("float32")
x_test = x_test[ ... , tf.newaxis].astype("float32")
print(x_train.shape)

(60000, 28, 28, 1)


In [16]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)
).shuffle(10000).batch(32) 

test_ds = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)
).batch(32)



# batch and shuffle dataset
# # be rady to modeling

In [7]:
# building the keras model

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()


In [9]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()


In [10]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [14]:
# train the model

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, 
    model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

In [ ]:
# test the model 

@tf.function

def test_step(images, ladels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [24]:
EPOCHS = 5

for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        train_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {},        Test Accuracy: {}'

    print(template.format(epoch + 1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))


Epoch 1, Loss: 0.005529128015041351, Accuracy: 99.80999755859375, Test Loss: 0.0,        Test Accuracy: 0.0
Epoch 2, Loss: 0.003781953826546669, Accuracy: 99.87000274658203, Test Loss: 0.0,        Test Accuracy: 0.0
Epoch 3, Loss: 0.003857559524476528, Accuracy: 99.87999725341797, Test Loss: 0.0,        Test Accuracy: 0.0
Epoch 4, Loss: 0.003763402346521616, Accuracy: 99.86000061035156, Test Loss: 0.0,        Test Accuracy: 0.0
Epoch 5, Loss: 0.00191799970343709, Accuracy: 99.95428466796875, Test Loss: 0.0,        Test Accuracy: 0.0
